In [75]:
import pandas as pd
death_rate = pd.read_csv('deaths/death-rates.csv')
population = pd.read_csv('census/data/table1_zcta.csv')
population = population_per_zipcode[['Zip Code', 'Population']]
income = pd.read_csv('income/data/returns-aig.csv')[::7]
income = income[['Zip Code', 'Returns', 'AIG']].reset_index()
pills = pd.read_csv('arcos-ca/pills-per-zipcode.csv')
merge = pd.merge(pills, income, how='left', on='Zip Code')
del merge['index']
merge = pd.merge(merge, population, how='left', on='Zip Code')
merge = pd.merge(merge, death_rate, how='left', on='Zip Code')
merge = merge.dropna()
merge = merge.sort_values(by=['Pills'], ascending=False).head(1000)
merge = merge[merge['Death Rate'] > 0]
merge['Returns'] = merge['Returns'].str.replace(',', '').astype('int64')
merge['AIG'] = merge['AIG'].str.replace(',', '').astype('int64')
merge['Population'] = merge['Population'].astype('int64')
merge.head()

,Zip Code,City,Pills,Returns,AIG,Population,Death Rate
0,94550,LIVERMORE,52022690,21341,2066480,46896,1.63
1,90242,DOWNEY,21791650,17767,726095,43497,2.65
2,92010,CARLSBAD,10548200,6737,495599,14382,12.55
3,95350,MODESTO,6547370,20938,891634,51046,2.19
4,95825,SACRAMENTO,6087880,13817,591794,31084,2.75


In [76]:
population = pd.DataFrame({
    'ZipCode': merge['Zip Code'],
    'City': merge['City'],
    'Pills per Person per Day': merge['Pills'] / merge['Population'] / 365,
    'AGI per Return': merge['AIG'] * 1e3 / merge['Returns'],
    'Death Rate': merge['Death Rate']
})
population.to_csv('population.csv', index=False)